https://blog.quantinsti.com/heston-model/

# Heston 1993 Model

## Vanilla Option 

In [1]:
import numpy as np

def heston_charfunc(u: np.ndarray, params: dict) -> np.ndarray:
    """
    Fonction caractéristique de Heston (1993) sous la mesure risque‑neutre.

    Parameters
    ----------
    u       : np.ndarray
        Fréquence de Fourier (réelle ou complexe).
    params  : dict
        Dictionnaire contenant :
            S0, r, q, T,
            v0, kappa, theta, sigma, rho.

    Returns
    -------
    np.ndarray
        Valeur de la fonction caractéristique φ(u).
    """
    S0   = params["S0"]
    r    = params["r"]
    q    = params["q"]
    T    = params["T"]
    v0   = params["v0"]
    kappa= params["kappa"]
    theta= params["theta"]
    sigma= params["sigma"]
    rho  = params["rho"]

    iu = 1j * u
    b  = kappa - rho * sigma * iu
    d  = np.sqrt(b**2 + sigma**2 * (u**2 + iu))
    g  = (b - d) / (b + d)

    exp_dT = np.exp(-d * T)

    C = iu * (r - q) * T \
        + (kappa * theta / sigma**2) * ((b - d) * T
           - 2.0 * np.log((1.0 - g * exp_dT) / (1.0 - g)))

    D = (b - d) / sigma**2 * ((1.0 - exp_dT) / (1.0 - g * exp_dT))

    return np.exp(C + D * v0 + iu * np.log(S0))  # pas de facteur e^{-qT} ici


def heston_price_call_fft(params: dict, N: int = 10_000, U_max: float = 1_000.0):
    """
    Prix call et put européens par la méthode FFT de Heston (1993).

    Parameters
    ----------
    params : dict
        Même dictionnaire que pour `heston_charfunc` + clé "K".
    N      : int
        Nombre de pas (sera forcé pair pour la règle de Simpson).
    U_max  : float
        Limite supérieure d'intégration sur la fréquence.

    Returns
    -------
    (call, put) : tuple[float, float]
    """
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    # règle de Simpson : N doit être pair
    if N % 2 == 1:
        N += 1

    u   = np.linspace(1e-10, U_max, N + 1)      # éviter u=0
    du  = u[1] - u[0]
    lnK = np.log(K)

    phi_u     = heston_charfunc(u, params)
    phi_u_im1 = heston_charfunc(u - 1j, params)
    phi_im1   = heston_charfunc(-1j, params)

    integrand_P1 = np.real(
        np.exp(-1j * u * lnK) * phi_u_im1 / (1j * u * phi_im1)
    )
    integrand_P2 = np.real(
        np.exp(-1j * u * lnK) * phi_u / (1j * u)
    )

    # poids de Simpson (1,4,2,4,2,...,4,1)
    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    P1 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P1)
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P2)

    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    put  = K * np.exp(-r * T) * (1.0 - P2) - S0 * np.exp(-q * T) * (1.0 - P1)

    return call, put


In [2]:
# Example
params = {
    "S0": 100,
    "K" : 65,
    "r":  0.02,
    "q": 0.01,
    "v0": 0.08,
    "kappa": 1.12,
    "theta": 0.05,
    "sigma": 0.50,
    "rho": -0.25,
    "T": 1.2
}

call, put = heston_price_call_fft(params)
print(f"Call Price: {call:.4f}")
print(f"Put Price: {put:.4f}")

Call Price: 36.3387
Put Price: 0.9901


## Extract Data before calibrating the model

In [3]:
import pandas as pd

excel_file = "market_data.xlsx"

wb = pd.read_excel(excel_file,sheet_name='NDX-2014' ,header=None,)


header_row_idx = 6
start_col = 2
end_col = 13               
n_maturities = 14

raw_header = wb.iloc[header_row_idx, start_col:end_col].tolist()
raw_data   = wb.iloc[
    header_row_idx+1 : header_row_idx+1+n_maturities,
    start_col : end_col
]


df = pd.DataFrame(
    raw_data.values[:, 1:],          
    index=raw_data.values[:, 0],    
    columns=raw_header[1:],         
)


df.columns = [f"{x:.1%}" for x in df.columns.astype(float)]


print(df.head())

S0 = wb.iat[3, 4]

r = wb.iat[1, 10]

q = wb.iat[2, 10]

       80.0%     90.0%     95.0%      97.5%   100.0%     102.5%    105.0%  \
T   3036.592  3416.166  3605.953  3700.8465  3795.74  3890.6335  3985.527   
1M     25.29     19.15     15.35       13.7    12.25      11.04     10.71   
2M     24.28     18.46     15.81      14.58    13.43      12.39     11.67   
3M     23.03     18.08     15.96      14.96     14.0       13.1     12.38   
6M     21.93     18.24     16.61      15.85    15.13      14.47     13.86   

      110.0%    120.0%  
T   4175.314  4554.888  
1M     11.46     13.56  
2M     11.52     12.55  
3M     11.83     12.06  
6M     12.87      12.4  


## Calibration

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.stats import norm

# Black–Scholes call price pour reconstruire les prix marchés à partir des vols implicites

def bs_call_price(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


# Conversion des labels de maturités en années
def maturity_to_T(label):
    if label == 'T':
        return None
    if label.endswith('M'):
        return int(label[:-1]) / 12.0
    if label.endswith('Y'):
        return float(label[:-1])
    raise ValueError(label)

# Construction d’une liste de points marché (T, K, prix_call)
market_data = []
strikes_abs = df.loc['T']  
for mat in df.index:
    T = maturity_to_T(mat)
    if T is None:
        continue
    for pct in df.columns:
        K      = strikes_abs[pct]
        vol_imp= df.loc[mat, pct]/100.0  # en pourcentage
        price  = bs_call_price(S0, K, r, q, vol_imp, T)
        market_data.append((T, K, price))

market_data = np.array(market_data, 
                       dtype=[('T', float), ('K', float), ('price', float)]) #Surface de prix


#3) Fonction objectif pour l’optimiseur 

def pack_params(x):
    """Dépaquète le vecteur x en dictionnaire de paramètres Heston."""
    return {
        "S0":   S0,
        "r":    r,
        "q":    q,
        "v0":   x[0],
        "kappa":x[1],
        "theta":x[2],
        "sigma":x[3],
        "rho":  x[4],
    }

def residuals(x):
    """Retourne les écarts modèle – marché pour chaque point de la surface."""
    params_base = pack_params(x)
    res = []
    for T, K, mkt_price in market_data:
        params = params_base.copy()
        params.update({"T": T, "K": K})
        model_call, _ = heston_price_call_fft(params, N=1000, U_max=2000)
        res.append(model_call - mkt_price)
    return np.array(res)


#4) Calibration par moindres carrés non‑linéaires 

# 4.1) Initial guess and bounds
x0 = np.array([
    0.04,    # v0
    1.0,     # kappa
    0.04,    # theta
    0.5,     # sigma
    -0.5,    # rho
])

# on impose rho ∈ [−0.999, 0.999] et les autres ≥ 0
lb = [1e-4, 1e-4, 1e-4, 1e-4, -0.999]
ub = [2.0, 10.0, 2.0, 5.0, 0.999]

opt = least_squares(
    residuals,
    x0,
    bounds=(lb, ub),
    verbose=2,
    xtol=1e-6,
    ftol=1e-6,
    max_nfev=200
)

# 4.2) Résultats
v0_fit, kappa_fit, theta_fit, sigma_fit, rho_fit = opt.x
print("Paramètres calibrés Heston :")
print(f" v0    = {v0_fit:.4f}")
print(f" kappa = {kappa_fit:.4f}")
print(f" theta = {theta_fit:.4f}")
print(f" sigma = {sigma_fit:.4f}")
print(f" rho   = {rho_fit:.4f}")


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.7768e+05                                    5.20e+07    
       1              2         2.2992e+04      2.55e+05       1.26e-01       8.48e+06    
       2              4         1.0201e+04      1.28e+04       1.07e-01       1.32e+06    
       3              5         5.6077e+03      4.59e+03       1.51e-01       1.07e+05    
       4              6         4.3447e+03      1.26e+03       1.34e-01       1.47e+06    
       5              7         3.6600e+03      6.85e+02       1.59e-01       1.66e+06    
       6              8         2.6891e+03      9.71e+02       7.51e-02       2.82e+05    
       7              9         2.3299e+03      3.59e+02       1.83e-01       1.20e+06    
       8             10         1.7595e+03      5.70e+02       4.01e-02       3.04e+05    
       9             11         1.6473e+03      1.12e+02       6.28e-02       5.46e+05    

In [5]:
# Test de la calibration
# On compare les prix Heston et Black-Scholes
print("Prix Black-Sholes, M=1Y, K=100%:", bs_call_price(S0,S0,r,q,0.1421,1.0))
params = {
    "S0": 100,
    "K" : 100,
    "r":  r ,
    "q": q ,
    "v0": v0_fit,
    "kappa": kappa_fit,
    "theta": theta_fit,
    "sigma": sigma_fit,
    "rho": rho_fit,
    "T": 1.0
}

print("Prix Heston, M=1Y, K=100%:", heston_price_call_fft(params, N=10000, U_max=2000)[0])

Prix Black-Sholes, M=1Y, K=100%: 210.03818079413418
Prix Heston, M=1Y, K=100%: 6.283248893033594


# Binary Option 

In [6]:
def heston_price_binary(S0,K,T,params, N=10000, U_max=1000):
    """
    Calculate binary option prices using the Heston model with FFT
    
    Parameters:
    params (dict): Model parameters
    N (int): Number of integration points
    U_max (float): Upper integration limit
    
    Returns:
    float: Binary option price
    """
    params["S0"] = S0
    params["K"]  = K
    params["T"]  = T
    r  = params["r"]

    if N % 2 == 1:
        N += 1

    u = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]
    lnK = np.log(K)

    # Calculate characteristic function
    phi_u = heston_charfunc(u, params)

    # For binary options, we only need P2
    integrand = np.real(np.exp(-1j * u * lnK) * phi_u / (1j * u))

    # Simpson's weights
    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    # Probability calculation
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand)

    # Binary option payoffs
    binary_call = np.exp(-r * T) * P2
    binary_put = np.exp(-r * T) * (1 - P2)

    return binary_call, binary_put



In [7]:
binary_call,binary_put = heston_price_binary(100,95,1,params)
print(f"Binary Call Price: {binary_call:.4f}")
print(f"Binary Put Price: {binary_put:.4f}")

Binary Call Price: 0.6567
Binary Put Price: 0.3294
